In [33]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [34]:
## Loading the model
from langchain_groq import ChatGroq
model = ChatGroq(model = 'Gemma2-9b-It', groq_api_key = groq_api_key)

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, My name is Diwakar Tiwari and working as a backend developer")])

AIMessage(content="Hello Diwakar Tiwari!\n\nIt's nice to meet you.  \n\nBackend development is a fascinating field. What kind of technologies do you primarily work with?  \n\nI'm always eager to learn more about different aspects of software development.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 24, 'total_tokens': 78, 'completion_time': 0.098181818, 'prompt_time': 0.002126046, 'queue_time': 0.232345153, 'total_time': 0.100307864}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9a6eb3a5-a285-43e4-988c-33faf24b93ee-0', usage_metadata={'input_tokens': 24, 'output_tokens': 54, 'total_tokens': 78})

In [ ]:
### Model can remember the previous messages
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hello, My name is Diwakar Tiwari and working as a backend developer"),
        AIMessage(content="Hello Diwakar Tiwari!\n\nIt's nice to meet you.  \n\nBackend development is a fascinating field. What kind of technologies do you primarily work with?  \n\nI'm always eager to learn more about different aspects of software development.\n"),
        HumanMessage(content="Hey, Whats my name?")
    ]
)

AIMessage(content='Your name is Diwakar Tiwari.  \n\nI remember it from our first conversation! 😊  How can I help you today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 92, 'total_tokens': 123, 'completion_time': 0.056363636, 'prompt_time': 0.004290191, 'queue_time': 0.232782458, 'total_time': 0.060653827}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8ef8ecbe-3a6e-4b2e-9cfa-93e626b06d30-0', usage_metadata={'input_tokens': 92, 'output_tokens': 31, 'total_tokens': 123})

In [21]:
## For particular chat there should be a chat session
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)
    

In [22]:
config = {"configurable": {"session_id":"chat1"}}

In [24]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, My name is Diwakar Tiwari")],
    config=config
)

response.content

"Hello Diwakar Tiwari, it's nice to meet you! \n\nWhat can I do for you today? 😊  \n"

In [31]:
## After changing the session_id from chat1 to chat2, now AI don't have access of chat1 history
config1 = {"configurable": {"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hello, My name is John")],
    config=config1
)

response.content

"Hello John! It's nice to meet you. 👋  \n\nHow can I help you today? 😊  \n\n"

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, what is my name?")],
    config=config1
)

response.content

'You told me your name was John earlier!  \n\nDo you need me to remember anything else? 😊\n'